In [ ]:
import nltk
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer

import numpy as np
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener

import json
import sys
#import langid
import re

import emoji
import pandas as pd

%matplotlib inline
from matplotlib import pyplot as plt

consumer_key = 'OrBHhvBsTwtzKPIOJ0XFjCCLw'
consumer_secret = 'aoisrrqSka7ZirLjjwM5A4y0y4ZySuA62C782WSpEBANLKN603'
access_token = '996280787971391489-n3wdQOJe6SflAqDbutSlhRQi7FoDfjU'
access_secret = 'q17dGaIzpLj3FLQeE10dqSpJa7LN0z2RpR0bPtZOEY556'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

# do not want retweet
def analyze_status(text):
    if 'RT' in text[0:3]:
        return(True)
    else:
        return(False)


# get the data
class MyStreamListener(StreamListener):
    tweet_number = 0
    def on_status(self, status):
        if 'sex' not in status.text and 'Sex' not in status.text and not analyze_status(status.text):
            with open('tweets.txt','a') as tf:
                tf.write(str(status.text))
                self.tweet_number += 1
        else:
            pass
        
        if self.tweet_number>=5000:
            sys.exit(1)
    def on_error(self, status):
        print("Error Code : " + status)

hashlist = ['#TikTok', '#tiktok', 'TikTok', 'tiktok', 'TIKTOK'] 


myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener , tweet_mode='extended')


myStream.filter(languages=['en'],track=hashlist)

In [ ]:
tweets = []
with open("tweets.txt") as f:
    # read each line
    flines = f.readlines()
    for line in flines:
        #tweets.append({"text":line})
        tweets.append(line)


pd.set_option('display.max_colwidth', -1)
pd.reset_option('display.max_colwidth')
pd.options.display.max_colwidth = 1000

import re
import functools
import operator

from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.corpus import wordnet
import nltk
import emoji


def get_wordnet_pos(word):
    # Map POS tag to first character lemmatize()
#     print("test")
    tag = nltk.pos_tag([word])[0][1][0].upper()
#     print("test1")
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

punctuation = punctuation + "…" + "‼"
stopwords_list = set(stopwords.words('english') + ["follow"])
porter = PorterStemmer()
lemmatizer = WordNetLemmatizer()


def clean_step1(tweet):
    tweet = re.sub(r'^RT\b', '', tweet) # remove RT at the begininig of sentense
    tweet = re.sub(r'(?:&[\w_]+;)', '', tweet) # remove &amp;
    tweet = tweet.lower() # convert text to lower-case
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', tweet) # remove URLs 
    tweet = re.sub(r'(?:#[\w_]+)', '', tweet)# remove hashtag
    tweet = re.sub(r'<.*?>', '', tweet) # remove HTML
    tweet = re.sub(r'(?:@[\w_]+)', '', tweet) # remove @-mentions 
    return tweet

def clean_step2(tweet):
    tweet = re.sub(r'(?:(?:\d+,?)+(?:\.?\d+)?)', '', tweet) # remove numbers
    #tweet = emoji.demojize(tweet)
    tweet = tweet.encode("ascii", errors="ignore").decode() # remove non-English words
    word_list = tweet.split(" ")
    word_list = [word for word in word_list if word not in stopwords_list] #remove stopwords
    tweet = " ".join(word_list)
    #tweet = emoji.emojize(tweet)
    return tweet

def clean_step3(tweet):
    word_list = tweet.split(" ")
    word_list = [porter.stem(word) for word in word_list] #stemming
    word_list = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in word_list if w != ""] #lemmatizer
    word_list = [word for word in word_list if len(word) > 1] #remove single-char words
    word_list = [word for word in word_list if word != "tiktok"] 
    tweet = " ".join(word_list)
    pattern = r"[{}]".format(punctuation) # create the punctuations pattern
    tweet = re.sub(pattern, "", tweet) 
    return tweet


processed_tweets=[]
list_tweets = []
    
for tweet in tweets:
    result = clean_step1(tweet)
    if result == "":
        continue
    list_tweets.append(result)
    processed_tweets.append({"text":result})
    
tweets = list_tweets
  
processed_tweets=[]
list_tweets = []
    
for tweet in tweets:
    result = clean_step2(tweet)
    if result == "":
        continue
    list_tweets.append(result)
    processed_tweets.append({"text":result})
    
tweets = list_tweets

processed_tweets=[]
list_tweets = []
    
for tweet in tweets:
    result = clean_step3(tweet)
    if result == "":
        continue
    list_tweets.append(result)
    processed_tweets.append({"text":result})
    
    
processed_tweets

cleaned_df = pd.DataFrame.from_dict(processed_tweets)


out_csv = []
for t in list_tweets:
    out_csv.append(t.split())
    
import csv

with open("tweets.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(out_csv)